In [2]:
#Importation of Libraries
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split  # Ajoutez cette ligne
import xgboost as xgb
from sklearn.metrics import accuracy_score
import nevergrad as ng

In [3]:
df = pd.read_csv('x_train_preprocessed.csv', header=0)
x_train_preprocessed = df.copy()
x_train_preprocessed= np.array(x_train_preprocessed)

In [4]:
y_train = pd.read_csv('y_train.csv', index_col=['id'])
labels_raw = y_train.copy() # a copy of the raw labels
labels_sorted = y_train.copy().sort_values(by = ['y']) # a copy of the raw labels sorted
#labels_raw.head()

In [5]:
data = np.array(df)
l = np.array(y_train)
labels = [l[i][0] for i in range(len(l))]
print (l)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [2]]


In [6]:
xtest = pd.read_csv('x_test_preprocessed.csv', header=0)
x_test_preprocessed = xtest.copy()
x_test_preprocessed= np.array(x_test_preprocessed)

In [7]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nevergrad as ng
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

def train_evaluate(hyperparameters):
    
    model = CatBoostClassifier(
        iterations=int(hyperparameters['iterations']),
        learning_rate=hyperparameters['learning_rate'],
        random_seed=int(hyperparameters['random_seed']),
        l2_leaf_reg=hyperparameters['l2_leaf_reg'],
        bagging_temperature = hyperparameters['bagging_temperature']
        
        #n_estimators=int(hyperparameters['n_estimators']),
        #max_depth=int(hyperparameters['max_depth']),
        
        #subsample=hyperparameters['subsample'],
        #colsample_bytree=hyperparameters['colsample_bytree']
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return -accuracy_score(y_test, preds)  

parametrization = ng.p.Dict(
    iterations = ng.p.Scalar(lower=10, upper=1000).set_integer_casting(),
    learning_rate = ng.p.Log(lower=0.01, upper=1),
    random_seed = ng.p.Scalar(lower=0, upper=1000).set_integer_casting(),
    l2_leaf_reg = ng.p.Scalar(lower=0, upper=1000),
    bagging_temperature = ng.p.Scalar(lower=0, upper=1000)
    #n_estimators=ng.p.Scalar(lower=10, upper=1000).set_integer_casting(),
    #max_depth=ng.p.Scalar(lower=3, upper=10).set_integer_casting(),
    #learning_rate=ng.p.Log(lower=0.01, upper=1),
    #subsample=ng.p.Scalar(lower=0.5, upper=1),
    #colsample_bytree=ng.p.Scalar(lower=0.5, upper=1)
)


optimizer = ng.optimizers.TwoPointsDE(parametrization=parametrization, budget=100)

for _ in range(optimizer.budget):
    x = optimizer.ask()
    loss = train_evaluate(x.value)
    optimizer.tell(x, loss)

# Meilleurs hyperparamètres trouvés
best_hyperparams = optimizer.recommend().value
print("Catboost:", best_hyperparams)

final_model = CatBoostClassifier(**best_hyperparams)
final_model.fit(data, y_train)
final_predictions = final_model.predict(x_test_preprocessed)


0:	learn: 1.3388012	total: 67.3ms	remaining: 1m 6s
1:	learn: 1.2979309	total: 69.4ms	remaining: 34s
2:	learn: 1.2626186	total: 71.3ms	remaining: 23.3s
3:	learn: 1.2319936	total: 73.4ms	remaining: 17.9s
4:	learn: 1.2053410	total: 75.2ms	remaining: 14.7s
5:	learn: 1.1820704	total: 77.2ms	remaining: 12.6s
6:	learn: 1.1717582	total: 351ms	remaining: 48.8s
7:	learn: 1.1521502	total: 354ms	remaining: 43.1s
8:	learn: 1.1349138	total: 357ms	remaining: 38.6s
9:	learn: 1.1197232	total: 360ms	remaining: 35s
10:	learn: 1.1063033	total: 362ms	remaining: 32s
11:	learn: 1.0944208	total: 366ms	remaining: 29.5s
12:	learn: 1.0838767	total: 369ms	remaining: 27.5s
13:	learn: 1.0745012	total: 372ms	remaining: 25.7s
14:	learn: 1.0661481	total: 375ms	remaining: 24.1s
15:	learn: 1.0586920	total: 378ms	remaining: 22.8s
16:	learn: 1.0520243	total: 381ms	remaining: 21.6s
17:	learn: 1.0460510	total: 384ms	remaining: 20.6s
18:	learn: 1.0406908	total: 387ms	remaining: 19.6s
19:	learn: 1.0380884	total: 417ms	remaini

CatBoostError: Length of label=4093 and length of data=5117 is different.

In [6]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nevergrad as ng
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

def train_evaluate(hyperparameters):
    
    model = XGBClassifier(
        n_estimators=int(hyperparameters['n_estimators']),
        max_depth=int(hyperparameters['max_depth']),
        learning_rate=hyperparameters['learning_rate'],
        subsample=hyperparameters['subsample'],
        colsample_bytree=hyperparameters['colsample_bytree']
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return -accuracy_score(y_test, preds)  

parametrization = ng.p.Dict(
    n_estimators=ng.p.Scalar(lower=10, upper=1000).set_integer_casting(),
    max_depth=ng.p.Scalar(lower=3, upper=10).set_integer_casting(),
    learning_rate=ng.p.Log(lower=0.01, upper=1),
    subsample=ng.p.Scalar(lower=0.5, upper=1),
    colsample_bytree=ng.p.Scalar(lower=0.5, upper=1)
)


optimizer = ng.optimizers.TwoPointsDE(parametrization=parametrization, budget=100)

for _ in range(optimizer.budget):
    x = optimizer.ask()
    loss = train_evaluate(x.value)
    optimizer.tell(x, loss)

# Meilleurs hyperparamètres trouvés
best_hyperparams = optimizer.recommend().value
print("Meilleurs hyperparamètres trouvés:", best_hyperparams)

final_model = XGBClassifier(**best_hyperparams)
final_model.fit(data, y_train)
final_predictions = final_model.predict(x_test_preprocessed)


Meilleurs hyperparamètres trouvés: {'n_estimators': 303, 'max_depth': 5, 'learning_rate': 0.09237223312265161, 'subsample': 0.5012201400142597, 'colsample_bytree': 0.7537687458263045}


XGBoostError: [01:21:54] /workspace/src/data/data.cc:501: Check failed: this->labels.Size() % this->num_row_ == 0 (4093 vs. 0) : Incorrect size for labels.
Stack trace:
  [bt] (0) /home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x3581ea) [0x7f257f7391ea]
  [bt] (1) /home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x389457) [0x7f257f76a457]
  [bt] (2) /home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x38a4b1) [0x7f257f76b4b1]
  [bt] (3) /home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(XGDMatrixSetInfoFromInterface+0xb0) [0x7f257f53f210]
  [bt] (4) /home/linuxbrew/.linuxbrew/opt/libffi/lib/libffi.so.8(+0x9056) [0x7f26024b4056]
  [bt] (5) /home/linuxbrew/.linuxbrew/opt/libffi/lib/libffi.so.8(+0x7ae9) [0x7f26024b2ae9]
  [bt] (6) /home/linuxbrew/.linuxbre